In [28]:
!pip install progress

  Created wheel for progress: filename=progress-1.5-py3-none-any.whl size=8078 sha256=f2f2fd934d6975e177a555fc9624b85a6f14c6c62678a2d442aade2f35444474

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.



  Stored in directory: c:\users\5uperpalo\appdata\local\pip\cache\wheels\4c\ff\85\0cabf2cb317421028ef98853ae5c8d84c31f3e4e11862ea977
Successfully built progress


In [1]:
!pip install alive-progress

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\tests\\__init__.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [ ]:
# downlad GAIN
!git clone git@github.com:jsyoon0823/GAIN.git
# upgrade GAIN to tensorflow v2, tf.placeholder is no longer wirking in v2 etc.
!tf_upgrade_v2 --infile GAIN/gain.py --outfile GAIN/gain_tf2.py
!tf_upgrade_v2 --infile GAIN/utils.py --outfile GAIN/utils_tf2.py
!pip install pyreadr

In [ ]:
# rename utils to .utils_tf2 in GAIN/gain_tf2.py
#from utils import normalization, renormalization, rounding
#from utils import xavier_init
#from utils import binary_sampler, uniform_sampler, sample_batch_index

In [1]:
import pandas as pd
import numpy as np
from GAIN.gain_tf2 import gain
from GAIN.utils_tf2 import binary_sampler
from GAIN.utils_tf2 import rmse_loss
import pyreadr

In [2]:
# dataset downloaded from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6C3JR1
train_normal_path = 'Tennessee_Event-Driven/datasets/TEP_FaultFree_Training.RData'
train_faulty_path = 'Tennessee_Event-Driven/datasets/TEP_Faulty_Training.RData'
test_normal_path = 'Tennessee_Event-Driven/datasets/TEP_FaultFree_Testing.RData'
test_faulty_path = 'Tennessee_Event-Driven/datasets/TEP_Faulty_Testing.RData'

train_normal_complete = pyreadr.read_r(train_normal_path)['fault_free_training']
train_faulty_complete = pyreadr.read_r(train_faulty_path)['faulty_training']
test_normal_complete = pyreadr.read_r(test_normal_path)['fault_free_testing']
test_faulty_complete = pyreadr.read_r(test_faulty_path)['faulty_testing']

In [3]:
# take only single run of the simulation
train_normal_complete = train_normal_complete[train_normal_complete["simulationRun"]==1]
train_faulty_complete = train_faulty_complete[train_faulty_complete["simulationRun"]==1]
test_normal_complete = test_normal_complete[test_normal_complete["simulationRun"]==1]
test_faulty_complete = test_faulty_complete[test_faulty_complete["simulationRun"]==1]

In [4]:
# drop columns that are not needed
train_normal_complete = train_normal_complete.drop(columns=["simulationRun","sample"])
train_faulty_complete = train_faulty_complete.drop(columns=["simulationRun","sample"])
test_normal_complete = test_normal_complete.drop(columns=["simulationRun","sample"])
test_faulty_complete = test_faulty_complete.drop(columns=["simulationRun","sample"])

In [5]:
tep_dataset = pd.concat([train_normal_complete, train_faulty_complete, test_normal_complete, test_faulty_complete], ignore_index=True, sort=False)

In [6]:
tep_dataset

,faultNumber,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0.0,0.25038,3674.0,4529.0,9.2320,26.889,42.402,2704.3,74.863,120.41,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,0.0,0.25109,3659.4,4556.6,9.4264,26.721,42.576,2705.0,75.000,120.41,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,0.0,0.25038,3660.3,4477.8,9.4426,26.875,42.070,2706.2,74.771,120.42,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,0.0,0.24977,3661.3,4512.1,9.4776,26.758,42.063,2707.2,75.224,120.39,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,0.0,0.29405,3679.0,4497.0,9.3381,26.889,42.650,2705.1,75.388,120.39,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30655,20.0,0.27389,3638.5,4512.7,9.3480,26.764,42.231,2713.2,74.348,120.43,...,54.051,26.807,63.206,21.214,36.950,39.891,48.834,49.969,42.244,16.814
30656,20.0,0.25405,3656.7,4490.2,9.3897,27.067,42.065,2715.9,74.945,120.40,...,54.334,25.185,61.865,21.657,37.441,41.566,50.203,50.270,41.264,17.765
30657,20.0,0.25664,3645.5,4506.3,9.4222,26.647,42.139,2715.5,74.313,120.41,...,54.141,24.931,63.582,20.960,37.131,36.523,42.912,50.497,41.720,20.372
30658,20.0,0.21993,3686.1,4507.0,9.3934,26.625,42.214,2710.9,74.022,120.40,...,53.535,21.874,61.885,20.723,38.850,42.909,47.052,50.774,40.902,20.166


In [7]:
tep_dataset.to_csv('Tennessee_Event-Driven/datasets/tep_extended_dataset_simrun1.csv',index=False)

# Create missing data in TEP dataset

In [3]:
tep_dataset = pd.read_csv('Tennessee_Event-Driven/datasets/tep_extended_dataset_simrun1.csv.csv',index_col=False)

In [4]:
dataset_X = tep_dataset.drop(columns=['faultNumber', 'simulationRun', 'sample']).values
dataset_Y = tep_dataset['faultNumber'].values

In [5]:
no, dim = dataset_X.shape
p = 0.1
# Introduce missing data
mask = binary_sampler(1-p, no, dim)
dataset_X_missing = dataset_X.copy()
dataset_X_missing[mask == 0] = np.nan

# Impute data

In [21]:
gain_parameters = {'batch_size': 128,
                 'hint_rate': 1.5,
                 'alpha': 100,
                 'iterations': 1000}

In [22]:
dataset_X_imputed = gain(dataset_X_missing, gain_parameters)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 168.12it/s]


In [23]:
rmse_loss(dataset_X, dataset_X_imputed, mask)

0.07301131265367386

# Pipeline

In [ ]:
class Pipeline:
    def __init__(self):
        pass
    
    def _impute(self):
        raise NotImplementedError()
    
    def _scale(self):
        raise NotImplementedError()
    
    def _classify(self):
        raise NotImplementedError()

    def process(self):
        raise NotImplementedError()

class FIREMAN_Pipeline(Pipeline):
    def __init__(self, dataset_x='', dataset_y='', imputer='', scaler='', classifier=''):
        self.task_id = task_id
        super().__init__()

    def _impute(self):
        raise NotImplementedError()
    
    def _scale(self):
        raise NotImplementedError()
    
    def _classify(self):
        raise NotImplementedError()

    def process(self):
        return

In [ ]:
pipe = Pipeline([('imputer',RandomOverSampler()), 
                 ('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier())])
#('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))